In [1]:
%load_ext lab_black
%cd ..

/mnt/h/hev/log-analytics


In [2]:
import argparse
import math
import multiprocessing
import sys
from datetime import datetime
from pathlib import Path
from pprint import pformat

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_optimizer
import yaml
from easydict import EasyDict
from pytorch_transformers import (
    BertForSequenceClassification,
    BertTokenizer,
    DistilBertForSequenceClassification,
    DistilBertTokenizer,
    RobertaForSequenceClassification,
    RobertaTokenizer,
)
from sklearn.model_selection import StratifiedKFold
from torch.optim import Adam, AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
from transformers import (
    AlbertForSequenceClassification,
    AlbertTokenizer,
    DebertaForSequenceClassification,
    DebertaTokenizer,
    SqueezeBertTokenizer,
    SqueezeBertForSequenceClassification,
    XLNetTokenizer,
    XLNetForSequenceClassification,
)

from datasets import load_test_data, load_train_data, MyDataset
from utils import SAM, AverageMeter, CustomLogger, FocalLoss, seed_everything

from main import MyTrainer
from collections import defaultdict
import matplotlib.pyplot as plt
import random

In [3]:
def get_dist(deck, feat, topk):
    dist = torch.norm(deck - feat[None], dim=1, p=None)
    knn = dist.topk(topk, largest=False)
    values, indices = knn
    return values, indices

In [4]:
with open("config/distilbert-base-uncased.yaml", "r") as f:
    C = EasyDict(yaml.load(f, yaml.FullLoader))
    C.result_dir = Path(C.result_dir)
    C.dataset.dir = Path(C.dataset.dir)
    seed_everything(C.seed, deterministic=False)

In [5]:
C

{'model': {'name': 'distilbert-base-uncased'},
 'comment': None,
 'result_dir': PosixPath('results/distilbert-base-uncased'),
 'debug': False,
 'seed': 20210425,
 'train': {'SAM': False,
  'folds': [1],
  'checkpoints': [None],
  'loss': {'name': 'focal', 'gamma': 2},
  'optimizer': {'name': 'AdamW'},
  'finetune': {'do': True, 'step1_epochs': 2, 'step2_epochs': 4},
  'max_epochs': 10,
  'lr': 1e-05,
  'scheduler': {'name': 'ReduceLROnPlateau',
   'params': {'factor': 0.5, 'patience': 3, 'verbose': True}}},
 'dataset': {'dir': PosixPath('data/ori'), 'batch_size': 30, 'num_workers': 8}}

In [6]:
trainer = MyTrainer(C, 1, "results/distilbert-base-uncased/distilbert-base-uncased-focal-AdamW-lr1e-05_1.pth")

Load pretrained results/distilbert-base-uncased/distilbert-base-uncased-focal-AdamW-lr1e-05_1.pth


In [7]:
model = trainer.model
model.eval()
torch.set_grad_enabled(False)

In [8]:
activation = []


def hook(model, input, output):
    activation.append(output.detach().cpu())

In [9]:
model.pre_classifier.register_forward_hook(hook)

## Train 데이터 deck 만들기

In [10]:
activation = []
tlevels = []
plevels = []
with tqdm(total=len(trainer.tdl.dataset), ncols=100, file=sys.stdout) as t:
    for id, text, tlevel, otext in trainer.tdl:
        tlevels.append(tlevel)
        plevel = model(text.cuda())[0].cpu()
        plevels.append(plevel)
        t.update(len(id))

100%|██████████████████████████████████████████████████████| 378377/378377 [19:32<00:00, 322.64it/s]


In [11]:
with tqdm(total=len(trainer.vdl.dataset), ncols=100, file=sys.stdout) as t:
    for id, text, tlevel, otext in trainer.vdl:
        tlevels.append(tlevel)
        plevel = model(text.cuda())[0].cpu()
        plevels.append(plevel)
        t.update(len(id))

100%|████████████████████████████████████████████████████████| 94595/94595 [04:52<00:00, 323.34it/s]


In [12]:
deck = {"feat": torch.cat(activation), "tlevel": torch.cat(tlevels), "fclevel": torch.cat(plevels)}

In [13]:
deck["feat"].shape, deck["tlevel"].shape, deck["fclevel"].shape

(torch.Size([472972, 768]), torch.Size([472972]), torch.Size([472972, 7]))

In [14]:
torch.save(deck, "results/distilbert-base-uncased/distilbert-base-uncased-focal-AdamW-lr1e-05_1-deck.pth")